## **Task 1 - Match user queries with resolved queries**

In [9]:
import pandas as pd

In [10]:
df1 = pd.read_csv('/content/new_queries.csv')
df1.head()

,Variation_Query,Matches_With_Query_ID
0,Unabel to conect to the internet,1
1,Can’t connect to internet,1
2,Intenet not working,1
3,Payment failed while chekout,2
4,Payment did not go through during chckout,2


In [11]:
df2 = pd.read_csv('/content/resolved_queries.csv')
df2.head()

,Query_ID,Pre_Resolved_Query
0,1,Unable to connect to the internet
1,2,Payment failed during checkout
2,3,App crashes when opening settings
3,4,Forgot password and unable to reset
4,5,Unable to upload files to the server


In [7]:
%pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.4 MB/s eta 0:00:00


In [18]:
import pandas as pd
import re
import string
from rapidfuzz import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)  # remove punctuation
    text = " ".join([lemmatizer.lemmatize(w) for w in text.split()])
    return text.strip()

df1["clean_query"] = df1["Variation_Query"].apply(preprocess)
df2["clean_query"] = df2["Pre_Resolved_Query"].apply(preprocess)

# ------------------------
# 2. Fuzzy Search Approach
# ------------------------
def get_best_fuzzy_match(query, choices, scorer=fuzz.token_sort_ratio, threshold=80):
    best_match, score, idx = process.extractOne(query, choices, scorer=scorer)
    if score >= threshold:
        return best_match, score
    else:
        return None, score

fuzzy_results = []
for q in df1["clean_query"]:
    best_match, score = get_best_fuzzy_match(q, df2["clean_query"], scorer=fuzz.token_set_ratio)
    fuzzy_results.append((q, best_match, score))

df1["Fuzzy_Match"], df1["Fuzzy_Score"] = zip(*[(m[1], m[2]) for m in fuzzy_results])

# Add Query_ID from df2 for fuzzy matches
id_map = dict(zip(df2["clean_query"], df2["Query_ID"]))
df1["Fuzzy_Match_ID"] = df1["Fuzzy_Match"].map(id_map)

# ------------------------
# 3. TF-IDF + Cosine Similarity
# ------------------------
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df2["clean_query"])  # resolved queries
query_vecs = vectorizer.transform(df1["clean_query"])        # unresolved

cosine_results = []
for i, q_vec in enumerate(query_vecs):
    cos_sim = cosine_similarity(q_vec, tfidf_matrix)
    best_idx = cos_sim.argmax()
    best_score = cos_sim[0, best_idx]
    cosine_results.append((df2.loc[best_idx, "clean_query"], best_score, df2.loc[best_idx, "Query_ID"]))

df1["Cosine_Match"], df1["Cosine_Score"], df1["Cosine_Match_ID"] = zip(*cosine_results)

# ------------------------
# 4. Final Output
# ------------------------
cols = ["Variation_Query",
        "Fuzzy_Match", "Fuzzy_Match_ID", "Fuzzy_Score",
        "Cosine_Match", "Cosine_Match_ID", "Cosine_Score"]

final_df = df1[cols]
print("\n✅ Matching Results:\n")
print(final_df)

# Save results
final_df.to_csv("/content/query_matching_results.csv", index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...



✅ Matching Results:

                              Variation_Query  \
0            Unabel to conect to the internet   
1                   Can’t connect to internet   
2                         Intenet not working   
3                Payment failed while chekout   
4   Payment did not go through during chckout   
5                  Payment issue at check out   
6    Application crashes when opening setings   
7            App crash when going to settings   
8            Settings cause the app to chrash   
9               Forgot passwrd and cant reset   
10        Forgotten password, unable to reset   
11                  I can’t reset my password   
12             Unable to uplod file to server   
13        Can't upload files on to the server   
14       File uploading to server not working   
15        No internet connection, please help   
16          Checkout page says payment failed   
17      Settings page crashes app immediately   
18            Password reset link not working  

## **Task 2 -  Match names**

In [19]:
df3 = pd.read_csv('/content/base_names.csv')
df3.head()

,Base_Name_ID,Base_Name
0,1,John Smith
1,2,Jennifer Brown
2,3,Michael O'Connor
3,4,Maria Garcia
4,5,Robert Lee


In [20]:
df4 = pd.read_csv('/content/name_variations.csv')
df4.head()

,Variation,Matches_With_Base_Name
0,Thomas King,Thomas King
1,ThomasKing,Thomas King
2,Maria Garcia,Maria Garcia
3,MaryLewis,Mary Lewis
4,Nancy W.,Nancy Wright


In [21]:
import pandas as pd
import re
from rapidfuzz import process, fuzz


# ------------------------
# 1. Preprocessing
# ------------------------
def clean_name(name):
    if pd.isnull(name):
        return ""
    name = str(name).lower()
    name = re.sub(r"[^a-z\s]", " ", name)  # keep only letters and spaces
    name = re.sub(r"\s+", " ", name).strip()  # normalize spaces
    return name

df3["clean_name"] = df3["Base_Name"].apply(clean_name)
df4["clean_name"] = df4["Variation"].apply(clean_name)

# ------------------------
# 2. Fuzzy Matching Function
# ------------------------
def best_name_match(query, choices, scorer=fuzz.token_set_ratio, threshold=80):
    best_match, score, _ = process.extractOne(query, choices, scorer=scorer)
    if score >= threshold:
        return best_match, score
    else:
        return None, score

matches = []
for var in df4["clean_name"]:
    match, score = best_name_match(var, df3["clean_name"], scorer=fuzz.token_sort_ratio, threshold=75)
    matches.append((match, score))

df4["Fuzzy_Match"], df4["Fuzzy_Score"] = zip(*matches)

# ------------------------
# 3. Add Base_Name_ID
# ------------------------
id_map = dict(zip(df3["clean_name"], df3["Base_Name_ID"]))
df4["Base_Name_ID"] = df4["Fuzzy_Match"].map(id_map)

# ------------------------
# 4. Final Output
# ------------------------
final_df = df4[["Variation", "Fuzzy_Match", "Base_Name_ID", "Fuzzy_Score"]]
print("\n✅ Name Matching Results:\n")
print(final_df)

# Save results
final_df.to_csv("/content/task2_name_matches.csv", index=False)



✅ Name Matching Results:

          Variation     Fuzzy_Match  Base_Name_ID  Fuzzy_Score
0      Thomas  King     thomas king          15.0   100.000000
1        ThomasKing            None           NaN    57.142857
2      Maria Garcia    maria garcia           4.0   100.000000
3         MaryLewis            None           NaN    52.631579
4          Nancy W.            None           NaN    73.684211
..              ...             ...           ...          ...
95  Jennifer- Brown  jennifer brown           2.0   100.000000
96    Daniel- Scott    daniel scott          17.0   100.000000
97         David M.            None           NaN    66.666667
98      Paul Allen.      paul allen          13.0   100.000000
99      Paul  Allen      paul allen          13.0   100.000000

[100 rows x 4 columns]
